In [1]:
!pip install trax

import trax
import trax.layers as tl
from trax.supervised.training import TrainTask,EvalTask,Loop
import pandas as pd

from nltk.tokenize import word_tokenize
import numpy as np
import random as rnd
from trax.supervised.training import Loop,TrainTask,EvalTask
import trax.fastmath.numpy as fastnp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 637.9/637.9 kB 9.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 50.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.4.0 requires cupy-cuda11x<12.0.0a0,>=9.5.0, which is not installed.
onnx 1.13.1 requires protobuf<4,>=3.20.2, but you have protobuf 3.19.6 which is incompatible.
kfp 1.8.20 requires google-api-python-client<2,>=1.7.8, but you have google-api-python-client 2.86.0 which is incompatible.
kfp 1.8.20 requires PyYAML<6,>=5.3, but you have pyyaml 6.0 which is incompatible.
gcsfs 2023.3.0 requires fsspec==2023

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
data=pd.read_csv(r'/kaggle/input/first-quora-dataset/q_quora.csv')

data.head()

/tmp/ipykernel_31/4072098737.py:1: DtypeWarning: Columns (7,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv(r'/kaggle/input/first-quora-dataset/q_quora.csv')


,id,qid1,qid2,question1,question2,is_duplicate,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404351 entries, 0 to 404350
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            404351 non-null  int64  
 1   qid1          404351 non-null  int64  
 2   qid2          404351 non-null  int64  
 3   question1     404350 non-null  object 
 4   question2     404349 non-null  object 
 5   is_duplicate  404351 non-null  object 
 6   Unnamed: 6    337 non-null     object 
 7   Unnamed: 7    27 non-null      object 
 8   Unnamed: 8    8 non-null       object 
 9   Unnamed: 9    3 non-null       object 
 10  Unnamed: 10   2 non-null       object 
 11  Unnamed: 11   2 non-null       object 
 12  Unnamed: 12   2 non-null       float64
dtypes: float64(1), int64(3), object(9)
memory usage: 40.1+ MB


In [4]:
duplicate_question=data[data['is_duplicate']=='1'].iloc[:,3:5]
len(duplicate_question)

149267

In [5]:
#duplicate_question=duplicate_question[:100000]

In [6]:
duplicate_question.head()

,question1,question2
5,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan..."
7,How can I be a good geologist?,What should I do to be a great geologist?
11,How do I read and find my YouTube comments?,How can I see all my Youtube comments?
12,What can make Physics easy to learn?,How can you make physics easy to learn?
13,What was your first sexual experience like?,What was your first sexual experience?


In [7]:
duplicate_question.isna().sum()

question1    0
question2    0
dtype: int64

In [8]:
duplicate_question.duplicated().sum()

0

In [9]:
train_size=int(len(duplicate_question)*.8)
val_size=int(len(duplicate_question)*.9)

train_data=duplicate_question[:train_size]
val_data=duplicate_question[train_size:val_size]


In [10]:
test_data=data.iloc[:,3:6][0:1000]
test_data.is_duplicate.value_counts()


0                                                                                                                                                       618
1                                                                                                                                                       380
Do bullets travel faster than the speed of sound when shot from a gun? If not, is it possible? If they do, what gun and how much devastation occurs?      1
My sister told my crush that I like her without me asking her to do so. What should I do now?                                                             1
Name: is_duplicate, dtype: int64

In [11]:
test_data=test_data[test_data.is_duplicate.isin(['0','1'])]
x_test=test_data.iloc[:,0:2]
y_test=test_data.is_duplicate.astype(int)

In [12]:
x_test.shape,y_test.shape

((998, 2), (998,))

In [13]:
print('Train data shape is :',train_data.shape)
print('Val data shape is :',val_data.shape)
print('Test data shape is :',test_data.shape)

Train data shape is : (119413, 2)
Val data shape is : (14927, 2)
Test data shape is : (998, 3)


In [14]:
def get_vovap(data):
    vocab={'<pad>':0,'<unknow>':1}
    question1=data['question1']
    question2=data['question2']
    for question in question1:
        words=word_tokenize(question)
        for word in words:
            if word not in list(vocab):
                vocab[word]=len(vocab)
    for question in question2:
        words=word_tokenize(question)
        for word in words:
            if word not in list(vocab):
                vocab[word]=len(vocab)
    return vocab

In [15]:
vocab=get_vovap(train_data[0:int(len(train_data)*.7)])
len(vocab)

32056

In [16]:
def get_tensor(question):
    tensor1=[]
    tensor2=[]
    question1=word_tokenize(question[0])
    question2=word_tokenize(question[1])
    for i in question1:
        tensor1.append(vocab.get(i,1))
    for i in question2:
        tensor2.append(vocab.get(i,1))
    return tensor1,tensor2

In [17]:
x,y=get_tensor(train_data.iloc[0])
print('sample from question1:',x)
print('sample from question2:',y)

sample from question1: [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
sample from question2: [4, 993, 6, 3949, 7, 268, 8, 124, 6368, 11, 20499, 169, 7, 270, 33, 16, 372, 18, 19, 20, 21]


In [18]:
def generator(data):
    idx=0
    while True:
        if idx>=len(data):
            idx=0
        questions=data.iloc[idx]
        tensor1,tensor2=get_tensor(questions)
        idx+=1
        tensor1,tensor2=np.array(tensor1),np.array(tensor2)
        yield tensor1,tensor2 
        

In [19]:
x,y=next(generator(train_data))
x.shape,y.shape

((20,), (21,))

In [20]:
boundaries =  [8,   16,  32, 64, 128, 256, 512]
batch_sizes = [256,128, 64, 32, 16,    8,   4,  2]


generate_train_data=trax.data.BucketByLength(boundaries,batch_sizes,length_keys=[0,1])(generator(train_data))
generate_val_data=trax.data.BucketByLength(boundaries,batch_sizes,length_keys=[0,1])(generator(val_data))


In [21]:
x,y=next(generate_train_data)
x.shape,y.shape

((128, 16), (128, 16))

In [22]:
# UNQ_C3 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED FUNCTION: TripletLossFn
def TripletLossFn(v1, v2, margin=0.25):
    """Custom Loss function.

    Args:
        v1 (numpy.ndarray): Array with dimension (batch_size, model_dimension) associated to Q1.
        v2 (numpy.ndarray): Array with dimension (batch_size, model_dimension) associated to Q2.
        margin (float, optional): Desired margin. Defaults to 0.25.

    Returns:
        jax.interpreters.xla.DeviceArray: Triplet Loss.
    """
    ### START CODE HERE (Replace instances of 'None' with your code) ###
    # use fastnp to take the dot product of the two batches (don't forget to transpose the second argument)
    scores = fastnp.dot(v1,v2.T) # pairwise cosine sim   
    # calculate new batch size
    batch_size = len(scores)
    # use fastnp to grab all postive `diagonal` entries in `scores`
    positive = fastnp.diagonal(scores) # the positive ones (duplicates)
    # subtract `fastnp.eye(batch_size)` out of 1.0 and do element-wise multiplication with `scores`
    negative_zero_on_duplicate = scores * (1.0 - fastnp.eye(batch_size))
    # use `fastnp.sum` on `negative_zero_on_duplicate` for `axis= None
    mean_negative = fastnp.sum(negative_zero_on_duplicate,axis=1) / (batch_size - 1)
    # create a composition of two masks: 
    # the first mask to extract the diagonal elements, 
    # the second mask to extract elements in the negative_zero_on_duplicate matrix that are larger than the elements in the diagonal 
    mask_exclude_positives = (np.identity(batch_size)==1)|(negative_zero_on_duplicate > positive.reshape(batch_size,1))
    # multiply `mask_exclude_positives` with 2.0 and subtract it out of `negative_zero_on_duplicate`
    negative_without_positive = negative_zero_on_duplicate - 2.0 * mask_exclude_positives
    # take the row by row `max` of `negative_without_positive`. 
    # Hint: negative_without_positive.max(axis = [?])  
    closest_negative = negative_without_positive.max(axis=1)
    # compute `fastnp.maximum` among 0.0 and `A`
    # where A = subtract `positive` from `margin` and add `closest_negative`
    # IMPORTANT: DO NOT create an extra variable 'A'
    triplet_loss1 = fastnp.maximum(0,margin - positive + closest_negative)
    # compute `fastnp.maximum` among 0.0 and `B`
    # where B = ubtract `positive` from `margin` and add `mean_negative`
    # IMPORTANT: DO NOT create an extra variable 'B'
    triplet_loss2 = fastnp.maximum(0,margin - positive + mean_negative)
    # add the two losses together and take the `fastnp.sum` of it    
    triplet_loss = fastnp.sum(triplet_loss1 + triplet_loss2)
    ### END CODE HERE ###
    
    return triplet_loss

In [23]:
v1 = np.array([[ 0.26726124,  0.53452248,  0.80178373],[-0.5178918 , -0.57543534, -0.63297887]])
v2 = np.array([[0.26726124, 0.53452248, 0.80178373],[0.5178918 , 0.57543534, 0.63297887]])
print("Triplet Loss:", TripletLossFn(v1,v2))

Triplet Loss: 0.7035077


In [24]:
from functools import partial
def TripletLoss(margin=0.25):
    triplet_loss_fn = partial(TripletLossFn, margin=margin)
    return tl.Fn('TripletLoss', triplet_loss_fn)

In [25]:
def normalize(x):  # normalizes the vectors to have L2 norm 1
        return x / fastnp.sqrt(fastnp.sum(x * x, axis=-1, keepdims=True))
    

In [26]:
def model(d_model):
    model=tl.Serial([tl.Embedding(len(vocab),d_model),
               tl.LSTM(d_model),
               tl.Mean(axis=1),
               tl.Fn('Normalize',lambda x:normalize(x))
               
    ])
    siamese=tl.Parallel(model,model)
    return(siamese)

In [27]:
print(model(512))

Parallel_in2_out2[
  Serial[
    Embedding_32056_512
    LSTM_512
    Mean
    Normalize
  ]
  Serial[
    Embedding_32056_512
    LSTM_512
    Mean
    Normalize
  ]
]


In [28]:
def learning_tasks(model,n_step):
    training=TrainTask(generate_train_data,loss_layer=TripletLoss(),
                       optimizer=trax.optimizers.Adam(0.001),
                       n_steps_per_checkpoint=100,)
    
    evaltion=EvalTask(generate_val_data,
                      metrics=[TripletLoss()])
    
    trainig_loop=Loop(model,training,eval_tasks=[evaltion],
                      output_dir='/kaggle/working/model')
    
    
    trainig_loop.run(n_step)
    return(trainig_loop)

In [29]:
Siamese=model(512)
n_step=int((train_size/32))*5
print(n_step)
model=learning_tasks(Siamese,n_step)

18655


/opt/conda/lib/python3.10/site-packages/jax/_src/xla_bridge.py:658: UserWarning: jax.host_count has been renamed to jax.process_count. This alias will eventually be removed; please update your code.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trax/layers/base.py:851: FutureWarning: GzipFile was opened for writing, but this will change in future Python releases.  Specify the mode argument for opening it for writing.
  with gzip.GzipFile(fileobj=f, compresslevel=compresslevel) as gzipf:



Step      1: Total number of trainable weights: 18511872
Step      1: Ran 1 train steps in 10.89 secs
Step      1: train TripletLoss |  15.74986649


/opt/conda/lib/python3.10/site-packages/trax/supervised/training.py:1249: FutureWarning: GzipFile was opened for writing, but this will change in future Python releases.  Specify the mode argument for opening it for writing.
  with gzip_lib.GzipFile(fileobj=f, compresslevel=2) as gzipf:


Step      1: eval  TripletLoss |  31.99929810

Step    100: Ran 99 train steps in 28.24 secs
Step    100: train TripletLoss |  22.25253677
Step    100: eval  TripletLoss |  10.21340752

Step    200: Ran 100 train steps in 19.82 secs
Step    200: train TripletLoss |  16.36700630
Step    200: eval  TripletLoss |  15.97921753

Step    300: Ran 100 train steps in 19.95 secs
Step    300: train TripletLoss |  14.20250607
Step    300: eval  TripletLoss |  14.24788475

Step    400: Ran 100 train steps in 21.00 secs
Step    400: train TripletLoss |  12.17187691
Step    400: eval  TripletLoss |  5.31993961

Step    500: Ran 100 train steps in 21.50 secs
Step    500: train TripletLoss |  9.83489037
Step    500: eval  TripletLoss |  8.57252979

Step    600: Ran 100 train steps in 21.59 secs
Step    600: train TripletLoss |  8.13150692
Step    600: eval  TripletLoss |  7.35132790

Step    700: Ran 100 train steps in 21.95 secs
Step    700: train TripletLoss |  7.08531952
Step    700: eval  TripletL

In [30]:
def cos_similarity(v1,v2):
    similarity=np.matmul(v1,v2.T)
    return similarity

In [31]:
def predict(model,question,threshold=.7):
    
    result=''
    
    question1,question2=get_tensor(question)
    
    question1,question2=np.expand_dims(question1,axis=0),np.expand_dims(question2,axis=0)
    
    v1,v2=model.eval_model([question1,question2])
    
    similarity=cos_similarity(v1,v2)[0][0]
    print('The Similarity is:',similarity)
    
    result='Similarity' if similarity>threshold else 'DisSimilarity'
    
    return result    

In [32]:
question=['How are you doing','how are you']
question1,question2=get_tensor(question)
predict(model,question)

The Similarity is: 0.7493155


'Similarity'

In [33]:
question=['what your name','how old are you']
predict(model,question)

The Similarity is: 0.08664571


'DisSimilarity'

In [34]:
question=['What is the story of Kohinoor (Koh-i-Noor) Dia...','What would happen if the Indian government sto...']
predict(model,question)

The Similarity is: 0.19641636


'DisSimilarity'

In [35]:
generate_test_data=trax.data.BucketByLength(boundaries=[512],batch_sizes=[len(x_test)],length_keys=[0,1])(generator(x_test))
x_test=next(generate_test_data)

In [36]:
def compute_accuracy(x_test,y_test,threshold=.7):
    accuracy=0
    y_test=y_test.values
    v1,v2=model.eval_model(x_test)
    for idx,(vector1,vector2) in enumerate(zip(v1,v2)):
        value=0
        similarity=cos_similarity(vector1,vector2)
        value=1  if similarity>threshold else 0
    
            
        if value ==y_test[idx]:
            accuracy+=1
    return(accuracy/len(y_test))
    
    
    
    

In [37]:
print('The Accurcy is:',compute_accuracy(x_test,y_test))

The Accurcy is: 0.7555110220440882
